# Sportsbetting data analysis 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
file_path = "data_analyst_task.xlsx"
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['Data Dictionary', 'Customer Bet Data', 'Customer Data']


In [3]:
# Load individual sheets to dataframes 
data_dict = pd.read_excel(xls, sheet_name='Data Dictionary')
bets_df = pd.read_excel(xls, sheet_name='Customer Bet Data')
customers_df = pd.read_excel(xls, sheet_name='Customer Data')

In [4]:
# Printing shape to get overview/ confirm correct loading
print("\n=== SHAPE ===")
print(f"Data Dictionary shape: {data_dict.shape}")
print(f"Customer Bet Data shape: {bets_df.shape}")
print(f"Customer Data shape: {customers_df.shape}")


=== SHAPE ===
Data Dictionary shape: (20, 2)
Customer Bet Data shape: (765299, 11)
Customer Data shape: (40000, 5)


## Reading in additional information on datasets 

In [5]:
pd.set_option('display.max_colwidth', None)
display(data_dict.head(20))
pd.reset_option('display.max_colwidth') # Returning to normal for later reading in

,Column Name,Description
0,month,"The month that a bet was settled, format YYYY-MM"
1,account_id,The unique identifier for an account
2,sport_name,The name of the sport a bet was placed on
3,in_play_yn,Y/N flag indicating whether a bet was placed on an event that was in-play at the time of placement
4,freebet_used_yn,Y/N flag indicating whether at least part of the amount placed on the bet was from a free bet
5,bet_type,"The type of bet placed. Single - only one selection included in the bet, Multiple - more than one selection included in the bet"
6,max_bet_yn,Y/N flag indicating whether the amount placed on the bet was >80% of the maximum amount allowed. The maximum value can differ by customer
7,market_group,"The market type a bet was placed on, this differs by sport"
8,stakes,Total monetary amount placed (in GBP)
9,revenue,Total revenue earned by the business (in GBP)


## Checking customer data integrity 

In [6]:
customers_df.head()

,account_id,marketing_channel,account_open_datetime,residence_country,sharp_yn
0,1,SEO,2015-03-19 18:41:41,Ireland,N
1,2,PPC (inc Competitor),2018-04-14 11:13:03,United Kingdom,N
2,3,SEO,2015-04-11 09:35:37,United Kingdom,N
3,4,Direct,2016-08-21 15:01:47,United Kingdom,N
4,5,SEO,2006-10-05 19:25:48,Ireland,N


In [7]:
print("\nCustomer Data dtypes:")
print(customers_df.dtypes)


Customer Data dtypes:
account_id                        int64
marketing_channel                object
account_open_datetime    datetime64[ns]
residence_country                object
sharp_yn                         object
dtype: object


In [8]:
print("\nCustomer Data missing values:")
print(customers_df.isnull().sum())


Customer Data missing values:
account_id               0
marketing_channel        0
account_open_datetime    0
residence_country        0
sharp_yn                 0
dtype: int64


In [9]:
print("\nCustomer Data duplicate values:")
customers_df.duplicated().sum()


Customer Data duplicate values:


0

In [10]:
print("\nCustomer Data summary:")
print(customers_df.describe(include='all'))


Customer Data summary:
         account_id marketing_channel          account_open_datetime  \
count   40000.00000             40000                          40000   
unique          NaN                11                            NaN   
top             NaN            Direct                            NaN   
freq            NaN             13662                            NaN   
mean    20000.50000               NaN     2015-10-15 20:01:37.732000   
min         1.00000               NaN            1997-04-30 23:00:00   
25%     10000.75000               NaN  2014-04-05 14:40:55.750000128   
50%     20000.50000               NaN     2017-03-14 10:52:49.500000   
75%     30000.25000               NaN            2018-04-14 12:54:11   
max     40000.00000               NaN            2018-12-31 15:05:48   
std     11547.14972               NaN                            NaN   

       residence_country sharp_yn  
count              40000    40000  
unique                72        2  
top

### Initial notes and insights on customer data 

1. No duplicates or missing values found. 
2. Each row represents a unique customer account, with 40,000 altogether.
3. A wide range of opening dates are visible, beginning in 1997. Therefore, I have a mix of old and new customers, which could offer insight on betting tendencies, profitability etc. This might need segmenting by tenure, as most have joined in recent years.
4. The vast majority of customers are from the UK. I could compare UK vs non-UK customer activities (betting behaviour etc) but may have to filter out countries with exceedingly small amounts of users. 
5. On marketing channel, there are 11 options with direct being most common. Channel vs profitability etc could be interesting to explore. Could segment these by channel.
6. A quarter of users are marked as sharp. Their betting trends could be explored, such as what sports they bet on, whether their revenue per bet is lower etc.

## Additional processing on customer data 

In [11]:
# Checking the percentage count of users by country 
country_counts = customers_df['residence_country'].value_counts(normalize=True) * 100
print(country_counts)

residence_country
United Kingdom                                   81.5025
Ireland                                          14.9850
Russian Federation                                0.7425
Ukraine                                           0.4175
Sweden                                            0.2500
                                                  ...   
British Virgin Islands                            0.0025
Cameroon                                          0.0025
Qatar                                             0.0025
Nepal                                             0.0025
Macao, Special Administrative Region of China     0.0025
Name: proportion, Length: 72, dtype: float64


In [12]:
# Keeping the specific country if it is UK or Ireland
# Otherwise, collating them as 'Other' due to the significantly low values in the dataset

customers_df['country_group'] = customers_df['residence_country'].apply(lambda x: x if x in ['United Kingdom', 'Ireland'] else 'Other')

country_group_counts = customers_df['country_group'].value_counts(normalize=True) * 100
print(country_group_counts)

country_group
United Kingdom    81.5025
Ireland           14.9850
Other              3.5125
Name: proportion, dtype: float64


The market is heaviy UK focused, with Ireland representing a significant minority. The 'other' countries likely have lower overall impact. 

Most analysis should focus on the UK, since it dominates the dataset. But Ireland, as a secondary market, deserves attention as there may be additional trends at play. Other may be excluded as the small size may make it difficult to spot trends. Additionally, as it is comprised of 70 countries, there are probably huge regional variations (among few players) at play.

In [18]:
# Checking what percentage of users from each country group are marked as sharp 
sharp_percent_by_country = customers_df.groupby("country_group")["sharp_yn"].apply(
    lambda x: (x == "Y").mean() * 100
).sort_values(ascending=False)

print(sharp_percent_by_country)

country_group
Other             93.736655
United Kingdom    25.048311
Ireland            8.625292
Name: sharp_yn, dtype: float64


The users from non-Ireland/UK countries are overwhelmingly sharp. This could be explored further: 
- may indicate targeted/fradulent accounts 
- sharp users may be utilising this platform from other jurisdictions 
- possible use of VPNs or similar 

Ireland and the UK make up the majority of users but show much lower sharp rates. Therefore, these may be the core recreational base. 

In [13]:
# Simplifying the exact timestamp value to respective columns for date and month, to allow for trend analysis without noise

customers_df['account_open_year'] = customers_df['account_open_datetime'].dt.year
customers_df['account_open_month'] = customers_df['account_open_datetime'].dt.month

print(customers_df[['account_open_datetime', 'account_open_year', 'account_open_month']].head())

  account_open_datetime  account_open_year  account_open_month
0   2015-03-19 18:41:41               2015                   3
1   2018-04-14 11:13:03               2018                   4
2   2015-04-11 09:35:37               2015                   4
3   2016-08-21 15:01:47               2016                   8
4   2006-10-05 19:25:48               2006                  10


In [14]:
# Exploring the spread of account opening dates (by year)

year_percentages = customers_df['account_open_year'].value_counts(normalize=True) * 100
print(year_percentages)

account_open_year
2018    38.0725
2017    14.9425
2016    11.3050
2015     7.0075
2014     6.5025
2012     4.2550
2013     4.2275
2011     3.4600
2010     2.7675
2009     1.6975
2008     1.4000
2007     1.0125
2006     1.0075
2005     0.5975
2002     0.4725
2004     0.4400
2003     0.3825
2001     0.2350
2000     0.1250
1997     0.0300
1998     0.0300
1999     0.0300
Name: proportion, dtype: float64


A large chunk of the accounts opened in 2018 (the year of the bets being placed). Therefore, there is a strong influx of new customers. 

Account openings drop steadily as you go back in years, indicating that older accounts are less active over time. 

The customer tenure is very skewed to new accounts. I could segment customers by tenure (2018 accounts vs earlier) to see if behaviour differs by how long they have been active. 

I could track growth trends; eg how did recent accounts perform as compared to existing customers. 

In [23]:
# Exploring the percentage of sharp accounts by opening year
counts = customers_df.groupby(['account_open_year', 'sharp_yn']).size().unstack(fill_value=0)
counts['sharp_percent'] = counts['Y'] / (counts['Y'] + counts['N']) * 100

print(counts['sharp_percent'])

account_open_year
1997    25.000000
1998    25.000000
1999    16.666667
2000    28.000000
2001    43.617021
2002    53.968254
2003    32.026144
2004    36.363636
2005    28.870293
2006    26.054591
2007    24.197531
2008    25.714286
2009    24.889543
2010    21.951220
2011    20.736994
2012    19.623972
2013    18.687167
2014    17.800846
2015    22.297538
2016    25.939850
2017    25.079471
2018    27.559262
Name: sharp_percent, dtype: float64


Sharpness isn't concentrated in one age group of accounts, there are peaks and troughs.

Overall, the older accounts have a higher percentage of sharp bettors. It may show that long-term customers are sharper (perhaps through learning betting strategies over time).

Recent accounts still show a relatively high percentage of sharps, so they may be also quite savvy (or sharp bettors are joining actively).

There is a dip in sharpness from 2010 - 2015, which might indicate a different user base signing up at that time. 

In [25]:
# Breaking down yearly sharp percentages by country group

customers_df['sharp_flag'] = customers_df['sharp_yn'].map({'Y': 1, 'N': 0})
sharp_by_country_year = customers_df.groupby(['country_group', 'account_open_year'])['sharp_flag'].mean() * 100

sharp_pivot = sharp_by_country_year.reset_index().pivot(index='account_open_year', columns='country_group', values='sharp_flag')
sharp_pivot = sharp_pivot.round(2)

print(sharp_pivot)

country_group      Ireland   Other  United Kingdom
account_open_year                                 
1997                 25.00     NaN             NaN
1998                 25.00     NaN             NaN
1999                 16.67     NaN             NaN
2000                 12.50     NaN           90.00
2001                  7.14     NaN           73.08
2002                 17.39     NaN           65.73
2003                  5.77     NaN           45.54
2004                 10.42    0.00           46.46
2005                  9.59     NaN           37.35
2006                  8.90  100.00           35.55
2007                  9.27     NaN           33.07
2008                 13.30  100.00           31.44
2009                  9.78  100.00           29.36
2010                  7.35   60.00           25.90
2011                  6.97  100.00           23.99
2012                  6.53   91.67           22.42
2013                 10.05   92.31           20.46
2014                  7.35   81

There's a clear downward trend in UK sharp percentage - either new UK accounts are less likely to be flagged or sharp bettors are utilising old accounts. 

Ireland has the lowest sharp percentages overall (so Irish sharp bettors are either less prevalent or not consistently flagged). In Ireland, like the UK, the average rate of sharpness is trending downwards on younger accounts. 

Consistent high rate of sharpness in the 'Other' group. 

In [15]:
# Checking the percentage count of marketing channels
channel_counts = customers_df['marketing_channel'].value_counts(normalize=True) * 100
print(channel_counts)

marketing_channel
Direct                  34.1550
Affiliates              18.4825
SEO                     16.3900
PPC Brand               15.5150
PPC (inc Competitor)     8.0775
Display                  2.9700
Social                   2.4525
Other                    1.5400
PPC ASA                  0.2625
RAE                      0.1450
PPC UAC                  0.0100
Name: proportion, dtype: float64


The largest share of customers are acquired directly (possibly indicating strong brand recognition?). Affiliates, SEO, paid search marketing and campaigns also seem profitable. 

From this, focus could be placed on the largest channels for retention/optimisation. Questions could be asked if smaller channels underperform (perhaps connecting here to revenue in the bets data)

In [20]:
# Checking sharp users by marketing channel 
sharp_by_channel = customers_df.groupby('marketing_channel')['sharp_yn'].value_counts(normalize=True).unstack().fillna(0)
sharp_by_channel['Sharp %'] = sharp_by_channel['Y'] * 100
print(sharp_by_channel[['Sharp %']])

sharp_yn                Sharp %
marketing_channel              
Affiliates            38.874611
Direct                23.034695
Display                9.511785
Other                 22.727273
PPC (inc Competitor)   8.882699
PPC ASA               13.333333
PPC Brand             21.028037
PPC UAC               25.000000
RAE                   36.206897
SEO                   30.109823
Social                12.640163


Affiliates, RAE and SEO seem to be attracting more sharp bettors. Risk management may consider marketing channel as a factor, since sharp bettors behave differently. 

In [22]:
pd.crosstab(index=customers_df['marketing_channel'], 
            columns=customers_df['country_group'], 
            values=customers_df['sharp_yn'].map({'Y': 1, 'N': 0}), 
            aggfunc='mean') * 100

country_group,Ireland,Other,United Kingdom
marketing_channel,,,
Affiliates,22.983871,92.608696,37.657267
Direct,8.078751,95.017182,23.248470
Display,5.442177,100.000000,10.549944
Other,12.698413,NaN,27.166276
PPC (inc Competitor),3.496503,90.909091,8.839779
PPC ASA,0.000000,100.000000,12.621359
PPC Brand,7.931034,94.117647,22.936642
PPC UAC,NaN,NaN,25.000000
RAE,0.000000,100.000000,29.545455


Again, the sharp concentration in 'Other' countries is standing out clearly. Across each marketing channel, sharp users are heavily overrepresented. Users from 'other' countries represent an acquisition risk - they're nearly all sharp, no matter the channel.

Overall, Ireland tends to have lower sharp rates, possibly indicating a more recreational user base.

UK is a mixed bag: some channels are attracting sharp behaviour and others are not.  

Channel-level marketing may need scrutiny due to the range in sharp rates seen (Affiliate, direct and SEO). It's possible that different regions need different marketing strategies.

## Checking customer bet data integrity

In [16]:
bets_df.head()

,month,account_id,sport_name,in_play_yn,freebet_used_yn,bet_type,max_bet_yn,market_group,stakes,revenue,betcount
0,2018-03,26928,Horse Racing,N,N,Single,N,Derivative Markets,9.282234,9.282234,1
1,2018-07,710,Golf,N,N,Single,N,Outright,8.715135,8.715135,1
2,2018-11,12592,Soccer,N,N,Multiple,N,Match Odds,9.395684,9.395684,2
3,2018-11,349,Soccer,N,N,Single,Y,Commercial Specials,18.583440,18.583440,1
4,2018-11,15999,Soccer,N,N,Single,N,Over/Under Markets,16.669625,8.334812,2


In [29]:
print("\nBet dtypes:")
print(bets_df.dtypes)


Bet dtypes:
month               object
account_id           int64
sport_name          object
in_play_yn          object
freebet_used_yn     object
bet_type            object
max_bet_yn          object
market_group        object
stakes             float64
revenue            float64
betcount             int64
dtype: object


In [30]:
print("\nBets missing values:")
print(bets_df.isnull().sum())


Bets missing values:
month              0
account_id         0
sport_name         0
in_play_yn         0
freebet_used_yn    0
bet_type           0
max_bet_yn         0
market_group       0
stakes             0
revenue            0
betcount           0
dtype: int64


In [36]:
print("\nBets duplicate values:")
bets_df.duplicated().sum()


Bets duplicate values:


0

In [31]:
print("\nBetting data summary:")
print(bets_df.describe(include='all'))


Betting data summary:
          month     account_id sport_name in_play_yn freebet_used_yn bet_type  \
count    765299  765299.000000     765299     765299          765299   765299   
unique       12            NaN         11          2               2        2   
top     2018-06            NaN     Soccer          N               N   Single   
freq      81714            NaN     433908     594515          662255   501362   
mean        NaN   19704.266411        NaN        NaN             NaN      NaN   
std         NaN   11417.043437        NaN        NaN             NaN      NaN   
min         NaN       1.000000        NaN        NaN             NaN      NaN   
25%         NaN   10125.000000        NaN        NaN             NaN      NaN   
50%         NaN   19655.000000        NaN        NaN             NaN      NaN   
75%         NaN   29352.000000        NaN        NaN             NaN      NaN   
max         NaN   40000.000000        NaN        NaN             NaN      NaN   

    

### Initial notes and insights on betting data 

1. Again, no missing or duplicate values. 
2. With the monthly data, could track betting trends across the calendar year (changes in sport popularity etc). 
3. 11 sport options, with soccer the most popular. Could explore which sports drive most revenue, which are riskiest. 
4. The yes/no values in in_play_yn, freebet_used_yn etc could explore whether in play, free bets etc are most profitable. Combining with the customer data, could explore what the 'sharp' users are utilising. 
5. The 49 different values in market_group could provide insight into which markets are profitable/ unprofitable (and this could tie back in to sport).
6. Stakes and revenue: could explore what is the distribution of this. Perhaps are some high-value customers driving higher revenue? 

With the scale of this dataset, it might need aggregating by account_id in order to provide useful insights. 